Benchmark: MatrixElement
========
This benchmark compares the evaluation time of the matrix element for all dists and all spins over a set of kinematical points. It includes the evaluation of physArea and the calculation of all momenta, spinors and resulting currents.

In [ ]:
from __future__ import print_function
import sftrident as sf
import sftrident.coordinate as corr
from sftrident.matrixElement2 import matrixElementWF
import numpy as np
import time
timesNP = {}
timesSQ = {}

Preliminary build
======

In [ ]:
def buildArr_lcc(Nss,N1m,N1x,N1y,N2m,N2x,N2y,eps=1e-5):
    ss = np.linspace(3.1,4.1,Nss)
    p1m = np.linspace(eps,1.5,N1m)
    p1x = np.linspace(-2,2,N1x)
    p1y = np.linspace(-2,2,N1y)
    p2m = np.linspace(eps,1.5,N2m)
    p2x = np.linspace(-2,2,N2x)
    p2y = np.linspace(-2,2,N2y)
    return [ss,p1m,p1x,p1y,p2m,p2x,p2y]


mass = 1.0
Narr1 = (2,3,4,5,6,7,8)
frame = 'lab'
tempDPHI = 50.0
config = {'a0':1e-4,'mass':1.0,'xi':0.0,'dPhi':tempDPHI,'psBounds':[-tempDPHI,tempDPHI],'envelope':'cos','pulseOpt':['analytic'],'deg':2500,'wf':True,'system':'lab'}

arrLCC = buildArr_lcc(*Narr1)

start = time.time()
kinObj = corr.momenta(*arrLCC,mass = mass,frame = frame,coordSys='lcc')
end =  time.time() - start
timesNP['initMom']=end

start = time.time()
matObj = matrixElementWF(kinObj,**config)
wrappMat = lambda spn : np.asarray(matObj.bw(*spn) + matObj.c(*spn) - matObj.bwx(*spn) - matObj.cx(*spn))
end =  time.time() - start
timesNP['initMat'] = end


In [ ]:
spins = [(0, 0, 0, 0), (0, 0, 0, 1), (0, 0, 1, 0), (0, 0, 1, 1), (0, 1, 0, 0), (0, 1, 0, 1), (0, 1, 1, 0), (0, 1, 1, 1), (1, 0, 0, 0), (1, 0, 0, 1), (1, 0, 1, 0), (1, 0, 1, 1), (1, 1, 0, 0), (1, 1, 0, 1), (1, 1, 1, 0), (1, 1, 1, 1)]

start = time.time()
resGridNP = np.array(map(wrappMat,spins))
end =  time.time() - start
timesNP['mapMat'] =end

print(resGridNP.shape)

In [ ]:



start = time.time()
matObjSQ = sf.matrixElement(config)
def getMatSQ(n):
    kinPara = [kinObj.grid.ssGrid[n],kinObj.grid.lccGrid1[1][n], kinObj.grid.lccGrid1[2][n],kinObj.grid.lccGrid1[0][n],kinObj.grid.lccGrid2[1][n], kinObj.grid.lccGrid2[2][n],kinObj.grid.lccGrid2[0][n]]
    matObjSQ.setKin(kinPara)
    resOld = matObjSQ.evalMatrixElementpart()
    return np.asarray(resOld)
timesSQ['initMat'] = time.time() - start

start = time.time()
resGridSQ = np.array(map(getMatSQ,np.arange(kinObj.grid.size)))
timesSQ['mapMat'] = time.time() - start


In [ ]:
print("="*10,"numpy time","="*10)
t_np_max = 0.0
el_max=0
t_np_sum = 0.0
for el in timesNP.iterkeys():
    t_np= timesNP[el]
    if t_np> t_np_max:
        t_np_max = t_np
        el_max = el
    t_np_sum +=t_np
    print("\t",el,"%1.2e"%t_np,'(for %d points)'%(kinObj.grid.size))
    print("\t",el,"%1.2e"%(t_np/(float(kinObj.grid.size))),'(avg. per point)')
print("full time: %1.2e"%t_np_sum,'(for %d points)'%(kinObj.grid.size))
print("avg. time: %1.2e"%(t_np_sum/(float(kinObj.grid.size))))
print("max. time: %1.2e"%t_np_max,"(%1.2f percent)"%(t_np_max/t_np_sum*100.0),'-> %s'%el_max)
print()  
print("="*10,"serial time","="*10)
t_sq_max = 0.0
el_max=0
t_sq_sum = 0.0
for el in timesSQ.iterkeys():
    t_sq= timesSQ[el]
    if t_sq> t_sq_max:
        t_sq_max = t_sq
        el_max = el
    t_sq_sum +=t_sq
    print("\t",el,"%1.2e"%t_sq,'(for %d points)'%(kinObj.grid.size))
    print("\t",el,"%1.2e"%(t_sq/(float(kinObj.grid.size))),'(avg. per point)')
print("full time: %1.2e"%t_sq_sum,'(for %d points)'%(kinObj.grid.size))
print("avg. time: %1.2e"%(t_sq_sum/(float(kinObj.grid.size))))
print("max. time: %1.2e"%t_sq_max,"(%1.2f percent)"%(t_sq_max/t_sq_sum*100.0),'-> %s'%el_max)